# Deploy Varco LLM Model 13B IST Package from AWS Marketplace 




VARCO-LLM is NCSOFT’s large language model, which can be applied to develop various NLP-based AI services such as Q&A, chatbot, summarization, information extraction etc. VACRO-LLM, trained with public pre-training data and internally constructed high-quality Korean data, boasts the highest performance among the Korean LLMs of similar sizes that have been released to date (see https://ncsoft.github.io/ncresearch/ for evaluation results). Our models will continue to be updated and we will also release LLMs that support multiple languages or are fined-tuned to specific tasks. As VARCO-LLM is currently in beta service (15 Aug to 10 Sep 2023), usage fees will not be charged temporally for this period. For inquiries regarding further performance improvement or collaboration for service applications, please contact us via email (varco_llm@ncsoft.com).

This sample notebook shows you how to deploy [varco llm](https://aws.amazon.com/marketplace/pp/prodview-aout3755izrye)  using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package [listing page](https://aws.amazon.com/marketplace/pp/prodview-m3sqbtxwpacom?sr=0-5&ref_=beagle&applicationId=AWSMPContessa)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [2]:
model_package_arn = "arn:aws:sagemaker:us-west-2:594846645681:model-package/ncsoft-ko-varco-13b-ist-final-dbf0cc4de8bf39e8926e3aa10e15a7fd"

In [3]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")
bucket

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


'sagemaker-us-west-2-376278017302'

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [5]:
model_name = "varco-llm-13b-ist"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g4dn.12xlarge"
)
batch_transform_inference_instance_type = (
    "ml.g4dn.12xlarge"
)

### A. Create an endpoint

In [6]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
endpoint_name = model_name
endpoint_name

'varco-llm-13b-ist'

In [8]:
%store endpoint_name

Stored 'endpoint_name' (str)


### B. Create input payload

In [9]:
input = {"text":"우주의 탄생 원리를 알려줘."}

### C. Perform real-time inference

In [10]:
response = runtime.invoke_endpoint(
    EndpointName=model_name,
    ContentType=content_type,
    Accept="application/json",
    Body=json.dumps(input)
)

json.load(response["Body"])

{'result': ['### User:\n우주의 탄생 원리를 알려줘.\n\n### Assistant:\n우주의 탄생 원리는 빅뱅 이론이라고도 알려진 빅뱅이다. 빅뱅은 우주가 갑자기 매우 빠르게 팽창하는 사건이다. 이 사건은 우주가 생성되는 데 필요한 모든 물질과 에너지를 포함하고 있기 때문에 매우 중요한 사건이다.\n빅뱅 이론은1915년에 아인슈타인이 일반 상대성 이론을 발표하면서 시작되었다. 일반 상대성 이론은 우주의 모든 물질과 에너지는 서로 다른 시공간에 존재한다는 것을 보여준다. 이것은 우주가 팽창한다는 것을 의미한다.\n빅뱅 이론은 우주의 탄생 원리를 이해하는 데 도움이 된다. 우주는 매우 작은 크기이지만, 매우 빠른 속도로 팽창하고 있다. 이것은 우주가 매우 작고, 매우 빠르게 팽창하고 있다는 것을 의미한다. 이것은 우주가 매우 크고, 매우 느리게 팽창하고 있다는 것을 의미한다.\n빅뱅 이론은 우주의 탄생 원리를 이해하는 데 도움이 되지만, 우주가 어떻게 시작되었는지에 대한 정확한 답을 제공하지는 않는다.']}

### D. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [11]:
"""
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)
"""

'\nmodel.sagemaker_session.delete_endpoint(model_name)\nmodel.sagemaker_session.delete_endpoint_config(model_name)\n'

## 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see this link:
1. [Batch trasfrom job documents](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html)

In [12]:
# upload the batch-transform job input files to S3

"""
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name)
print("Transform input uploaded to " + transform_input)
"""

'\ntransform_input_folder = "data/input/batch"\ntransform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name)\nprint("Transform input uploaded to " + transform_input)\n'

In [13]:
# Run the batch-transform job
"""
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()
"""

'\ntransformer = model.transformer(1, batch_transform_inference_instance_type)\ntransformer.transform(transform_input, content_type=content_type)\ntransformer.wait()\n'

In [14]:
# output is available on following path

#transformer.output_path

## 4. Clean-up

### A. Delete the model

In [15]:
#model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

